In [1]:
import tensorflow as tf

n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [2]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')

In [10]:
import numpy as np

def neuron_layer(X, n_nuerons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_nuerons), stddev=stddev)
        W = tf.Variable(init, name ='kernel')
        b = tf.Variable(tf.zeros([n_nuerons]), name='bias')
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [11]:
with tf.name_scope('dnn'):
    hidden1 = neuron_layer(X, n_hidden1, name='hidden1', activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name='hidden2', activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name='outpus')

In [3]:
# using built in tf

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1', activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name='hidden2', activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name='outputs')

In [4]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')

In [5]:
learning_rate = 0.01

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    

In [6]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [7]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [9]:
from tensorflow.examples.tutorials.mnist import input_data
mnist= input_data.read_data_sets("/tmp/data/")

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [10]:
n_epochs = 40
batch_size = 50

In [14]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={ X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={ X: mnist.test.images, y: mnist.test.labels })
        
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Train accuracy: 0.94 Test accuracy: 0.901
1 Train accuracy: 0.92 Test accuracy: 0.9205
2 Train accuracy: 0.88 Test accuracy: 0.9287
3 Train accuracy: 0.98 Test accuracy: 0.9376
4 Train accuracy: 0.94 Test accuracy: 0.9427
5 Train accuracy: 0.96 Test accuracy: 0.9461
6 Train accuracy: 0.94 Test accuracy: 0.9501
7 Train accuracy: 0.96 Test accuracy: 0.9502
8 Train accuracy: 0.94 Test accuracy: 0.9539
9 Train accuracy: 0.92 Test accuracy: 0.9575
10 Train accuracy: 0.98 Test accuracy: 0.9594
11 Train accuracy: 0.96 Test accuracy: 0.9608
12 Train accuracy: 1.0 Test accuracy: 0.9617
13 Train accuracy: 0.98 Test accuracy: 0.9635
14 Train accuracy: 0.96 Test accuracy: 0.9638
15 Train accuracy: 0.98 Test accuracy: 0.9649
16 Train accuracy: 0.98 Test accuracy: 0.9676
17 Train accuracy: 0.94 Test accuracy: 0.9688
18 Train accuracy: 1.0 Test accuracy: 0.969
19 Train accuracy: 0.94 Test accuracy: 0.9699
20 Train accuracy: 0.98 Test accuracy: 0.9686
21 Train accuracy: 1.0 Test accuracy: 0.9706
22 

In [ ]:
with tf.Session() as sess: 
    saver.restore(sess, './my_model_final.ckpt')
    X_new_scaled = 